In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


****What is Vulngpt ?
-** This an application where user can Detect the Basic website Vulnerbility.**


# Step 1: Install and Import Libraries


In [ ]:
!pip install transformers

import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch


# **Step2: Load SQL Injection and Other Vulnerabilities Datasets**

In [ ]:
import pandas as pd
from sklearn.utils import shuffle

# Load the SQL Injection dataset
sql_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Train/sql_injection2.0.csv')
# Load the Other Vulnerabilities dataset (for XSS, CSRF, RCE)
other_vuln_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Train/Other_Vulnerabilities_Dataset2.0.csv')

# Shuffle the datasets
sql_data_shuffled = shuffle(sql_data)
other_vuln_data_shuffled = shuffle(other_vuln_data)

# Verify the datasets
print("SQL Injection Dataset:")
print(sql_data_shuffled.head())

print("\nOther Vulnerabilities Dataset:")
print(other_vuln_data_shuffled.head())

# Print dataset columns
print("\nSQL Injection Dataset Columns:", sql_data_shuffled.columns)
print("Other Vulnerabilities Dataset Columns:", other_vuln_data_shuffled.columns)


SQL Injection Dataset:
      ID                                          SQL Query Injection Payload  \
521  522         UPDATE accounts SET balance=500 WHERE id=1               NaN   
737  738  DELETE FROM orders WHERE order_id = john.doe@e...               NaN   
740  741         UPDATE accounts SET balance=500 WHERE id=2               NaN   
660  661     UPDATE accounts SET balance=500 WHERE id=apple               NaN   
411  412  SELECT email FROM customers WHERE email = '' O...           1=1;--'   

    Injection Type Vulnerability Status     Injection Result Attack Type  
521            NaN                   No                  NaN      Normal  
737            NaN                   No                  NaN      Normal  
740            NaN                   No                  NaN      Normal  
660            NaN                   No                  NaN      Normal  
411    Union-based                  Yes  Unauthorized access      Attack  

Other Vulnerabilities Dataset:
      ID

# Step 3: Tokenize Both Datasets Using CodeBERT

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

# Function to tokenize data from multiple columns and combine input_ids
def tokenize_data(dataframe, text_columns):
    # Assuming concatenation of texts from different columns for tokenization
    concatenated_texts = dataframe[text_columns].apply(lambda x: ' '.join(x.dropna().values), axis=1)
    return tokenizer(concatenated_texts.tolist(), padding="max_length", truncation=True, max_length=512, return_tensors='pt')

# Columns to tokenize - choose how you concatenate based on your model's capacity to handle input
sql_columns = ['SQL Query', 'Vulnerability Status']
other_vuln_columns = ['Attack Vector', 'Description']

# Tokenize data
sql_encoded = tokenize_data(sql_data, sql_columns)
other_vuln_encoded = tokenize_data(other_vuln_data, other_vuln_columns)

# Printing to verify the outputs
print("SQL Encoded Data:")
print(f"Input IDs: {sql_encoded['input_ids'].shape}")
print(f"Attention Masks: {sql_encoded['attention_mask'].shape}")

print("\nOther Vulnerabilities Encoded Data:")
print(f"Input IDs: {other_vuln_encoded['input_ids'].shape}")
print(f"Attention Masks: {other_vuln_encoded['attention_mask'].shape}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


SQL Encoded Data:
Input IDs: torch.Size([1000, 512])
Attention Masks: torch.Size([1000, 512])

Other Vulnerabilities Encoded Data:
Input IDs: torch.Size([1000, 512])
Attention Masks: torch.Size([1000, 512])


# Step 5: Create a PyTorch Dataset Class and Split Both Datasets for Training and Validation
This class converts the tokenized data into a format compatible with PyTorch’s Dataset.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}  # Correct way to handle tensor copying
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)
        #return len(self.labels)

# Assuming sql_encoded and other_vuln_encoded are already prepared using your tokenization function
class CodeDataset_other(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = [float(label) for label in labels]

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Assuming sql_encoded and other_vuln_encoded are already prepared using your tokenization function
sql_labels = [1 if x == 'Yes' else 0 for x in sql_data['Vulnerability Status']]
other_labels = [1 if x == 'Yes' else 0 for x in other_vuln_data['Vulnerability Status']]

# Create dataset instances
sql_dataset = CustomDataset(sql_encoded, sql_labels)
other_vuln_dataset = CodeDataset_other(other_vuln_encoded, other_labels)

# Example: Using DataLoader to handle batches
train_loader = DataLoader(sql_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(other_vuln_dataset, batch_size=16, shuffle=False)

# Example loop to illustrate how to handle data loader outputs
for batch in train_loader:
    print(batch)  # This should print the batches without warnings
    break


{'input_ids': tensor([[    0, 49179,  1047,  ...,     1,     1,     1],
        [    0, 49179,  1009,  ...,     1,     1,     1],
        [    0, 49179,  1047,  ...,     1,     1,     1],
        ...,
        [    0, 10089,  3850,  ...,     1,     1,     1],
        [    0, 34543,  2349,  ...,     1,     1,     1],
        [    0, 49179,  1009,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1])}


In [ ]:


# Assuming sql_dataset and other_vuln_dataset are your datasets prepared from previous steps
train_sql, val_sql = train_test_split(sql_dataset, test_size=0.1, random_state=42)
train_other, val_other = train_test_split(other_vuln_dataset , test_size=0.1, random_state=42)

NameError: name 'train_test_split' is not defined

In [ ]:
# prompt: Show the dataset size of each type in the other_vuln and sql injection csv
import pandas as pd # import the pandas library

# Load the SQL injection dataset.
sql_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Train/sql_injection2.0.csv') # Replace 'path/to/your/sql_injection_dataset.csv' with the actual path

# Load the other vulnerabilities dataset
other_vuln_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Train/Other_Vulnerabilities_Dataset2.0.csv') # Replace 'path/to/your/other_vulnerabilities_dataset.csv' with the actual path


# SQL Injection Dataset:
#    ID                                          SQL Query Injection Payload  \
# 0   1  UPDATE accounts SET balance=500 WHERE id=' UNI...               NaN
# 1   2               SELECT * FROM users WHERE id = '100'               NaN
# 2   3   SELECT * FROM us
# Assuming your dataset has a column named 'Vulnerability Type'
vulnerability_counts = other_vuln_data['Attack Type'].value_counts()

print("Vulnerability Type Counts:\n", vulnerability_counts)

Vulnerability Type Counts:
 Attack Type
CSRF    273
LFI     262
XSS     233
RCE     232
Name: count, dtype: int64


# Step 6: Initialize CodeBERT Models for Both Datasets****

In [ ]:
sql_model = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=2)
other_vuln_model = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=other_vuln_data['Vulnerability Status'].nunique())

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train

In [ ]:
!pip install seqeval



# IoU Calculation Function

In [ ]:
def calculate_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters:
    box1 : dict
        Keys: {'x1', 'y1', 'x2', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    box2 : dict
        Keys: {'x1', 'y1', 'x2', 'y2'}

    Returns:
    float
        in the range [0, 1]
    """
    # Determine the coordinates of the intersection rectangle
    x_left = max(box1['x1'], box2['x1'])
    y_top = max(box1['y1'], box2['y1'])
    x_right = min(box1['x2'], box2['x2'])
    y_bottom = min(box1['y2'], box2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The area of intersection
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # The area of both AABBs
    box1_area = (box1['x2'] - box1['x1']) * (box1['y2'] - box1['y1'])
    box2_area = (box2['x2'] - box2['x1']) * (box2['y2'] - box2['y1'])

    # Compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area.
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou


# compute_metrics Function

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)  # Assuming you're doing classification

    # Placeholder IoU calculation
    # You need to ensure that 'labels' and 'predictions' include bounding box info
    # This is a stub example assuming box data is available in your labels/predictions
    ious = [calculate_iou(pred_box, true_box) for pred_box, true_box in zip(predictions_boxes, labels_boxes)]
    mean_iou = sum(ious) / len(ious) if ious else 0

    # Calculate precision, recall, and F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')

    # Calculate AUC-ROC
    auc_roc = roc_auc_score(labels, predictions)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc_roc": auc_roc,
        "mean_iou": mean_iou  # Include mean IoU in the metrics returned
    }


In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import RobertaForSequenceClassification

# Define sql_model here
sql_model = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=2)
other_vuln_model = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=other_vuln_data['Vulnerability Status'].nunique())

# Setup training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
)

# Initialize the Trainer for SQL model
trainer_sql = Trainer(
    model=sql_model,
    args=training_args,
    train_dataset=train_sql,
    eval_dataset=val_sql,
    compute_metrics=compute_metrics  # Include custom compute_metrics function
)
trainer_other = Trainer(
    model=other_vuln_model,
    args=training_args,
    train_dataset=train_other,
    eval_dataset=val_other,
    compute_metrics=compute_metrics  # Include custom compute_metrics function
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train both models
trainer_sql.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000900,0.000250,1.000000,1.000000,1.000000,1.000000
2,0.000100,0.000048,1.000000,1.000000,1.000000,1.000000
3,0.000100,0.000026,1.000000,1.000000,1.000000,1.000000


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Description seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Attack Vector seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Description seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Attack Vector seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


TrainOutput(global_step=675, training_loss=0.07264020845397479, metrics={'train_runtime': 303.7848, 'train_samples_per_second': 8.888, 'train_steps_per_second': 2.222, 'total_flos': 710399849472000.0, 'train_loss': 0.07264020845397479, 'epoch': 3.0})

In [ ]:
trainer_other.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.013500,0.006133,0.000000,0.000000,0.000000,0.000000
2,0.004900,0.000040,0.000000,0.000000,0.000000,0.000000
3,0.004100,0.001669,0.000000,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Attack Vector seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Description seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


TrainOutput(global_step=675, training_loss=0.04582816220268055, metrics={'train_runtime': 307.1906, 'train_samples_per_second': 8.789, 'train_steps_per_second': 2.197, 'total_flos': 710393471078400.0, 'train_loss': 0.04582816220268055, 'epoch': 3.0})

In [ ]:
# Evaluate both models
sql_results = trainer_sql.evaluate()
other_results = trainer_other.evaluate()

# Print results
print("SQL Injection Model Results:", sql_results)
print("General Vulnerabilities Model Results:", other_results)


SQL Injection Model Results: {'eval_loss': 0.00011795405589509755, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 2.8968, 'eval_samples_per_second': 34.521, 'eval_steps_per_second': 4.488, 'epoch': 3.0}
General Vulnerabilities Model Results: {'eval_loss': 0.005357217974960804, 'eval_model_preparation_time': 0.0035, 'eval_accuracy': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 2.8541, 'eval_samples_per_second': 35.037, 'eval_steps_per_second': 4.555, 'epoch': 3.0}


RESULT of Train

RESULT of Train

In [ ]:
model_path_sql = './model_sql'
model_path_other = './model_other'
trainer_sql.model.save_pretrained(model_path_sql)
trainer_other.model.save_pretrained(model_path_other)

# Build a Scraping

Predicting the the vulnerbility

In [ ]:
!pip install transformers # Ensure transformers is installed.
from transformers import RobertaTokenizer, RobertaForSequenceClassification # Import the RobertaTokenizer and RobertaForSequenceClassification classes
import requests
from bs4 import BeautifulSoup
import re
import torch # Import torch

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')
sql_model = RobertaForSequenceClassification.from_pretrained('/content/model_sql')
other_vuln_model = RobertaForSequenceClassification.from_pretrained('/content/model_sql')


def scrape_webpage(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    html_content = str(soup)
    script_content = [script.text for script in soup.find_all('script') if script.text]
    hidden_inputs = soup.find_all('input', {'type': 'hidden'})
    data_attributes = [tag for tag in soup.find_all(attrs=re.compile(r'^data-'))]
    sql_candidates = []

    for input_tag in hidden_inputs:
        if 'sql' in input_tag.get('value', '').lower():
            sql_candidates.append(input_tag['value'])

    for data_tag in data_attributes:
        for attribute, value in data_tag.attrs.items():
            if 'sql' in attribute.lower() and isinstance(value, str):
                sql_candidates.append(value)

    return html_content, script_content, sql_candidates


def suggest_fixes(sql_prediction, other_vuln_prediction, script_content):
  """Provides code suggestions based on detected vulnerabilities."""
  suggestions = []
  if sql_prediction == 1:
    suggestions.append(
        "**SQL Injection Detected:**\n"
        "**Suggestion:** Sanitize user inputs before using them in SQL queries.\n"
        "**Example:** Use parameterized queries or escape special characters."
    )

  if other_vuln_prediction == 1:
    suggestions.append(
        "**Other Vulnerability (XSS, CSRF, RCE) Detected:**\n"
        "**Suggestion:** Investigate the script_content for potential vulnerable code.\n"
        "**Possible fixes:**\n"
        " - **XSS:** Encode user input before displaying it.\n"
        " - **CSRF:** Use anti-CSRF tokens.\n"
        " - **RCE:** Validate user input to prevent code execution."
    )


  if not suggestions:
    suggestions.append("No vulnerabilities detected.")

  return suggestions


def predict_and_suggest(url):
    """Predicts vulnerabilities and suggests fixes for a given URL."""
    html_content, script_content, sql_candidates = scrape_webpage(url) # scrape_webpage is now defined
    encoded_input = tokenizer(script_content, return_tensors="pt", truncation=True, padding=True)
    sql_outputs = sql_model(**encoded_input)
    other_vuln_outputs = other_vuln_model(**encoded_input)
    sql_predictions = torch.argmax(sql_outputs.logits, dim=-1)
    other_vuln_predictions = torch.argmax(other_vuln_outputs.logits, dim=-1)

    suggestions = suggest_fixes(sql_predictions.item(), other_vuln_predictions.item(), script_content)

    return suggestions


# Example Usage
url = "https://www.facebook.com/"  # Replace with the target URL
suggestions = predict_and_suggest(url)

for suggestion in suggestions:
  print(suggestion)

In [ ]:
!pip install transformers # Ensure transformers is installed.

In [ ]:
!pip install transformers # Ensure transformers is installed.
from transformers import RobertaTokenizer, RobertaForSequenceClassification # Import the RobertaTokenizer and RobertaForSequenceClassification classes

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')
sql_model = RobertaForSequenceClassification.from_pretrained('/content/model_sql')
other_vuln_model = RobertaForSequenceClassification.from_pretrained('/content/model_sql')


def suggest_fixes(sql_prediction, other_vuln_prediction, script_content):
  """Provides code suggestions based on detected vulnerabilities."""
  suggestions = []
  if sql_prediction == 1:
    suggestions.append(
        "**SQL Injection Detected:**\n"
        "**Suggestion:** Sanitize user inputs before using them in SQL queries.\n"
        "**Example:** Use parameterized queries or escape special characters."
    )

  if other_vuln_prediction == 1:
    suggestions.append(
        "**Other Vulnerability (XSS, CSRF, RCE) Detected:**\n"
        "**Suggestion:** Investigate the script_content for potential vulnerable code.\n"
        "**Possible fixes:**\n"
        " - **XSS:** Encode user input before displaying it.\n"
        " - **CSRF:** Use anti-CSRF tokens.\n"
        " - **RCE:** Validate user input to prevent code execution."
    )


  if not suggestions:
    suggestions.append("No vulnerabilities detected.")

  return suggestions


def predict_and_suggest(url):
    """Predicts vulnerabilities and suggests fixes for a given URL."""
    html_content, script_content, sql_candidates = scrape_webpage(url)
    encoded_input = tokenizer(script_content, return_tensors="pt", truncation=True, padding=True)
    sql_outputs = sql_model(**encoded_input)
    other_vuln_outputs = other_vuln_model(**encoded_input)
    sql_predictions = torch.argmax(sql_outputs.logits, dim=-1)
    other_vuln_predictions = torch.argmax(other_vuln_outputs.logits, dim=-1)

    suggestions = suggest_fixes(sql_predictions.item(), other_vuln_predictions.item(), script_content)

    return suggestions


# Example Usage
url = "https://www.facebook.com/"  # Replace with the target URL
suggestions = predict_and_suggest(url)

for suggestion in suggestions:
  print(suggestion)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


NameError: name 'scrape_webpage' is not defined